In [1]:
import math
import numpy
import random
import types
from itertools import izip, tee, imap
from operator import itemgetter
from cStringIO import StringIO
import tensorflow as tf

In [2]:
class ConvNet(Graph):
  IMAGE_PIXELS = 784
  NUM_CLASSES = 10
  def __init__(self):
    self.build(train=False)

  def build(self, keep_prob=1.0, lr = 1e-4):
    self.x = tf.placeholder(tf.float32, [None, self.IMAGE_PIXELS], name='x-input')
    self.y_ = tf.placeholder(tf.float32, [None, self.NUM_CLASSES], name='y-input')

    def weight_variable(shape):
      return tf.Variable(tf.truncated_normal(shape, stddev=0.1))
    def bias_variable(shape):
      return tf.Variable(tf.constant(0.1, shape=shape))
    def conv2d(x, W):
      return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
    def max_pool_2x2(x):
      return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    x_image = tf.reshape(self.x, [-1,28,28,1])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)

    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)

    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    b_fc1 = bias_variable([1024])
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

    W_fc2 = weight_variable([1024, self.NUM_CLASSES])
    b_fc2 = bias_variable([self.NUM_CLASSES])

    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    y = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

    y = tf.nn.softmax(tf.matmul(h_fc1, W_fc2) + b_fc2)

    # training
    self.loss = -tf.reduce_sum(self.y_*tf.log(y))
    self.train = tf.train.AdamOptimizer(lr).minimize(self.loss)

    # evaluation
    self.evaluate = tf.cast(tf.equal(tf.argmax(y,1), tf.argmax(self.y_,1)), tf.float32)

  def transform_batch(self, batch):
    x, y_ = iunzip(batch)
    return {
      self.x: list(x),
      self.y_: list(y_)
    }

class ConvNetEval(ConvNet):
  def __init__(self):
    self.build(keep_prob = 1.)

class ConvNetTrain(ConvNet):
  def __init__(self):
    self.build(
      keep_prob = .5,
      lr = 1e-4)

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

partitions = 8

train_data = sc\
  .parallelize(zip(mnist.train.images, mnist.train.labels), partitions)\
  .cache()     
test_data = sc\
  .parallelize(zip(mnist.test.images, mnist.test.labels), partitions)\
  .cache()

In [4]:
master_graph = tf.Graph()
with master_graph.as_default(), tf.Session() as session:
  dg = DistributedGraph(sc, session, partitions, 50, ConvNetEval)
  #dg.params = last_params
  for i in xrange(4):
    batched_train_data = dg.shuffle_and_batch(train_data, dg.partitions, dg.batch_size, 0.5)
    x = dg.train(batched_train_data, worker_epochs=2, graph_cls=ConvNetTrain)
    print 'Training loss: %.2f' % x
    if i == 0 or ((i+1) % 2 == 0):
      print 'Epoch %i Test Acc: %.3f' % (i, dg.evaluate(test_data, graph_cls=ConvNetEval))